# Moodle 3

Con esta guía de Jupyter vamos a lograr instalar una versión reciente de Moodle, en un servidor Ubuntu 16.04 recién instalado.

Primero debe instalar los componentes del servidor LAMP, inciciando por la instalación de la base de datos. Seguidamente descargamos el código de Moodle.

Por último, creamos la base de datos a ser utilizada por Moodle, y luego corremos el instalador Web.


### MariaDB
MySQL y MariaDB tienen algunas pequeñas diferencias pero para nuestro objetivo son técnicamente equivalente, por lo cual vamos a preferir MariaDB sobre MySQL.

A la hora de instalar la base de datos de MariaDB, la herramienta de Apt de Debian va a insistir en solicitarnos una contraseña para el usuario "root" de la base de datos, el cual vamos a utilizar para tareas de administración.

Para evitar que en esta guía se nos solicite contraseña, vamos a usar el comando ``debconf-set-selections`` para definir la contraseña de antemano.

Nota: El usuario "root" de MariaDB y el de Linux son completamente aparte.

In [ ]:
echo "mariadb-server mysql-server/root_password password greencore" | sudo debconf-set-selections
echo "mariadb-server mysql-server/root_password_again password greencore" | sudo debconf-set-selections
sudo apt-get -y install mariadb-server

## Configuración de MariaDB

Moodle además de tener capacidad para otros motores de bases de datos como PostgreSQL y Oracle, para obtener un mayor desempeño nos solicitan realizar un pequeño cambio en el tipo de tabla a utilizar.

Para ello debemos agregar las últimas 3 líneas de este ejemplo a la configuración de MariaDB, dentro de la sección **[mysqld]** dentro del archivo ``/etc/mysql/mariadb.conf.d/50-server.cnf``

```
[mysqld]

#
# * Basic Settings
#
user            = mysql
pid-file        = /var/run/mysqld/mysqld.pid
socket          = /var/run/mysqld/mysqld.sock
port            = 3306
basedir         = /usr
datadir         = /var/lib/mysql
tmpdir          = /tmp
lc-messages-dir = /usr/share/mysql
skip-external-locking

default_storage_engine = innodb
innodb_file_per_table = 1
innodb_file_format = Barracuda
```
Seguidamente, reiniciamos el servicio de MariaDB (llamado "mysql")

In [ ]:
sudo systemctl restart mysql
sudo systemctl status mysql | cat

### Apache y Php

Procedemos ahora a instalar Apache2, con el componente de Php el cual fuerza el servidor a usar el MPM **prefork**.

Así mismo instalamos módulos de Php requeridos por la aplicación Moodle, según la documentación del fabricante.

Nota: No vamos a reiniciar el servicio de Apache en este momento, sino hasta luego de editar archivos de configuración.

In [ ]:
sudo apt-get -y install php libapache2-mod-php apache2 graphviz aspell php-pspell php-curl php-gd php-intl \
 php-mysql php-xml php-xmlrpc php-ldap php-zip php-mbstring php-soap
sudo a2enmod rewrite

### Cambios menores en Apache

**Desde una terminal**, y utilizando su editor preferido, debe cambiar dos archivos de la siguiente forma:

**Archivo:** ``/etc/apache2/mods-enabled/dir.conf``

**Cambio:** La línea que inicia por DirectoryIndex tiene una lista de las extensiones de archivos de tipo "Index". Ya sea cambiando el orden, o eliminando extensiones, debe asegurarse que ``index.php`` esté de primero en la lista.

``DirectoryIndex index.php``

**Archivo:** ``/etc/apache2/apache2.conf``

**Cambio:** En la definición del directorio ``/var/www`` originalmente tiene la instrucción ``AllowOverride None``. Cambie la definición para que diga ``AllowOverride All``

``<Directory /var/www/>
        Options Indexes FollowSymLinks
        AllowOverride All
        Require all granted
</Directory>``

Reiniciamos Apache y mostramos el estado del servicio.

## Moodle

La última versión y recomendada de Moodle es la 3.3.1. Sin embargo vamos a instalar una versión anterior, para practicar procesos de actualización.

In [ ]:
sudo systemctl restart apache2
sudo systemctl status apache2 | cat
wget -qc https://download.moodle.org/download.php/direct/stable32/moodle-latest-32.tgz
ls -lh

Descomprimimos el código de Moodle dentro de /var/www/html que es el directorio default de Apache.

En un sistema en producción se recomienda crear un directorio para cada virtualhost, o un único virtualhost para Moodle y hacer uso de la función de "Multi tenant" para trabajar varios clientes con una sola instancia de la aplicación.

Importante que el código queda con derechos de **www-data**, usuario que utiliza Debian para el servicio de web de Apache. Esto porque Moodle debe poder manipular archivos dentro de la carpeta.

In [ ]:
sudo tar xzf ~/moodle-latest-32.tgz -C /var/www/
sudo rm -r /var/www/html
sudo mv /var/www/moodle /var/www/html
sudo chown -R www-data:www-data /var/www/html
ls /var/www/html

### Creación de base de datos

Antes de ejecutar el instalador Web de Moodle debemos crear la base de datos y el usuario que usará la aplicación de Moodle.

Para ello vamos a utilizar el comando de mysql(1), definiendo la contraseña "greencore" en la misma línea de comando.

In [ ]:
sudo mysql -u root -pgreencore << EOF
CREATE DATABASE moodle DEFAULT CHARACTER SET utf8 COLLATE utf8_unicode_ci;
CREATE USER 'moodle'@'localhost' IDENTIFIED BY 'greencore';
GRANT SELECT,INSERT,UPDATE,DELETE,CREATE,CREATE TEMPORARY TABLES,DROP,INDEX,ALTER ON moodle.* TO 'moodle'@'localhost' IDENTIFIED BY 'greencore';
EOF

## Configuración de Moodle

Para el caso de Moodle, hace falta un directorio adicional llamado ``/var/www/moodledata`` el cual debemos crear y asignar permisos para que Moodle pueda crear contenido en este lugar cuando sea necesario.

In [ ]:
sudo mkdir /var/www/moodledata
sudo chown -R www-data /var/www/moodledata
sudo chmod -R 0770 /var/www/moodledata

### Instalador web

No es necesario manipular ningún archivo Php para definir la configuración de nuestro sitio. Visitamos el sitio web, donde nos recibe el instalador Web de Moodle. Este funciona en estilo "wizard" donde simplemente vamos contestando con cuidado las consultas que nos hace.

http://10.42.25.6X/

**Importante**: Seleccione el "Database driver" de **MariaDB** en lugar de Improved Mysql.

En el instalador puede usar valores que usaría en un sitio en producción.
Recuerde utilizar para este laboratorio el usuario "admin", aunque no va a aceptar la contraseña "greencore" para facilitar la revisión de errores o dudas. Se recomienda **Gr33nc0r3.** para este laboratorio.